In [1]:
import sys
sys.path.append('..')
from iltransr.models.il_smiles_only import*

[03:20:28] ../src/storage/storage.cc:199: Using Pooled (Naive) StorageManager for CPU
[03:20:30] ../src/storage/storage.cc:199: Using Pooled (Naive) StorageManager for GPU


In [2]:
cnn_dropout = 0.1
batch_size = 64
bucket_num, bucket_ratio = 2, 0

In [3]:
net = ILNet(dropout=cnn_dropout)
net.encoder = model.encoder
net.src_embed =  model.src_embed
net.hybridize()
print(net)

ILNet(
  (src_embed): HybridSequential(
    (0): Embedding(72 -> 128, float32)
    (1): Dropout(p = 0.0, axes=())
  )
  (encoder): TransformerEncoder(
    (layer_norm): LayerNorm(eps=1e-05, axis=-1, center=True, scale=True, in_channels=128)
    (transformer_cells): HybridSequential(
      (0): TransformerEncoderCell(
        (attention_cell): MultiHeadAttentionCell(
          (_base_cell): DotProductAttentionCell(
            (_dropout_layer): Dropout(p = 0, axes=())
          )
          (proj_query): Dense(128 -> 128, linear)
          (proj_key): Dense(128 -> 128, linear)
          (proj_value): Dense(128 -> 128, linear)
        )
        (proj): Dense(128 -> 128, linear)
        (ffn): PositionwiseFFN(
          (ffn_1): Dense(128 -> 1024, linear)
          (activation): Activation(relu)
          (ffn_2): Dense(1024 -> 128, linear)
          (layer_norm): LayerNorm(eps=1e-05, axis=-1, center=True, scale=True, in_channels=128)
        )
        (layer_norm): LayerNorm(eps=1e-05, ax

In [4]:
save_dir = os.path.join(root_path,'iltransr/pre-trained params')

In [5]:
def get_dataloader(train_dataset,train_smiles_lengths):

    # Pad data, stack label and lengths
    batchify_fn = nlp.data.batchify.Tuple(
        nlp.data.batchify.Pad(axis=0, pad_val=0, ret_length=True),
        nlp.data.batchify.Stack(dtype='float32'))
    train_batch_sampler = nlp.data.sampler.FixedBucketSampler(
        train_smiles_lengths,
        batch_size=batch_size,
        num_buckets=bucket_num,
        ratio=bucket_ratio,
        shuffle=True)

    # Construct a DataLoader object for both the training and test data
    train_dataloader = gluon.data.DataLoader(dataset=train_dataset,
                                             batch_sampler=train_batch_sampler,
                                             batchify_fn=batchify_fn)
    return train_dataloader

In [6]:
import pandas as pd
MP_database = pd.read_excel(os.path.join(root_path,'datasets/il properties/IL-TOX.xlsx'),sheet_name='TOX')

In [7]:
train_smiles = MP_database['IL SMILES'].map(canonical_smile).map(no_split).map(preprocess)
train_smiles_lengths = MP_database['IL SMILES'].map(canonical_smile).map(no_split).map(get_length)
train_MP =mx.nd.array(MP_database['Experimental logEC50'])
train_dataset = gluon.data.SimpleDataset(gluon.data.ArrayDataset(train_smiles,train_MP))
train_dataloader = get_dataloader(train_dataset,train_smiles_lengths)

In [8]:
epochs, learning_rate = 150,0.001                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            
max_len = 100
train(net, train_dataloader, batch_size, learning_rate, ctx, epochs)

[03:20:34] ../src/operator/cudnn_ops.cc:292: Auto-tuning cuDNN op, set MXNET_CUDNN_AUTOTUNE_DEFAULT to 0 to disable
[03:20:35] ../src/operator/cudnn_ops.cc:292: Auto-tuning cuDNN op, set MXNET_CUDNN_AUTOTUNE_DEFAULT to 0 to disable


[Epoch 0] train avg loss 1.68263, train avg r2 -2294.75,throughput 0.0305223K wps
learning rate: 0.001
[Epoch 1] train avg loss 1.31523, train avg r2 -38.9618,throughput 0.246075K wps
learning rate: 0.001
[Epoch 2] train avg loss 0.89065, train avg r2 -13.7407,throughput 4.62232K wps
learning rate: 0.001
[Epoch 3] train avg loss 0.725765, train avg r2 -10.2348,throughput 4.58165K wps
learning rate: 0.001
[Epoch 4] train avg loss 0.711949, train avg r2 -3.37014,throughput 4.68123K wps
learning rate: 0.001
[Epoch 5] train avg loss 0.441734, train avg r2 0.398643,throughput 4.63341K wps
learning rate: 0.001
[Epoch 6] train avg loss 0.383446, train avg r2 0.558341,throughput 4.82K wps
learning rate: 0.001
[Epoch 7] train avg loss 0.317612, train avg r2 0.733567,throughput 4.49558K wps
learning rate: 0.001
[Epoch 8] train avg loss 0.274351, train avg r2 0.781917,throughput 4.7132K wps
learning rate: 0.001
[Epoch 9] train avg loss 0.300262, train avg r2 0.768128,throughput 4.73487K wps
learn

KeyboardInterrupt: 